<a href="https://colab.research.google.com/github/carolineb3/US-FlightDelayModeling/blob/main/Data_Cleaning_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

**Retrieve and Combine Datasets, Remove Cancelled/Diverted Flights**

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load and merge data
path = '/content/drive/MyDrive/MIS 545 Project/Raw Data Files'

may = pd.read_csv(path + '/May_Reports.csv')
june = pd.read_csv(path + '/June_Reports.csv')
july = pd.read_csv(path + '/July_Reports.csv')

df_raw = pd.concat([may, june, july], ignore_index=True)

# Remove cancelled and diverted flights out from dataset
df_clean = df_raw[(df_raw['CANCELLED'] == 0) & (df_raw['DIVERTED'] == 0)].copy()

print(f'Shape of dataset: {df_clean.shape}')
print(f'Columns: {df_clean.columns}')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Shape of dataset: (1809857, 27)
Columns: Index(['YEAR', 'MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'OP_UNIQUE_CARRIER',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'DEP_DELAY_NEW', 'DEP_TIME_BLK', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_TIME_BLK', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY'],
      dtype='object')


**Remove Leakage Columns and Feature Engineering**

In [ ]:
# Remove columns to prevent leakage
df_clean = df_clean.drop(columns=[
    "DEP_DELAY_NEW","ARR_TIME",
    "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY",
    "LATE_AIRCRAFT_DELAY","ACTUAL_ELAPSED_TIME"], errors='ignore')

# Feature engineering

# Convert FL_DATE to datetime
df_clean['FL_DATE'] = pd.to_datetime(df_clean['FL_DATE'],format='%m/%d/%Y %I:%M:%S %p',errors='coerce')
df_clean['MONTH'] = df_clean['FL_DATE'].dt.month
df_clean['DAY_OF_WEEK'] = df_clean['DAY_OF_WEEK'].astype(int)

# Convert CRS_DEP_TIME into HOUR
df_clean['CRS_DEP_TIME'] = pd.to_numeric(df_clean['CRS_DEP_TIME'], errors='coerce').fillna(0).astype(int)
df_clean['HOUR'] = df_clean['CRS_DEP_TIME'].astype(str).str.zfill(4).str[:2].astype(int)

# Categorical
df_clean['OP_UNIQUE_CARRIER'] = df_clean['OP_UNIQUE_CARRIER'].astype('category')
df_clean['ORIGIN'] = df_clean['ORIGIN'].astype('category')
df_clean['DEST'] = df_clean['DEST'].astype('category')
df_clean['DEP_TIME_BLK'] = df_clean['DEP_TIME_BLK'].astype('category')

# Frequency Encoding for ORIGIN & DEST

# Compute frequency of each airport in the full dataset
origin_freq = df_clean['ORIGIN'].value_counts()
dest_freq = df_clean['DEST'].value_counts()

# Add frequency encoded columns to df_clean
df_clean['ORIGIN_FREQ'] = df_clean['ORIGIN'].map(origin_freq)
df_clean['DEST_FREQ'] = df_clean['DEST'].map(dest_freq)

# Drop original high-cardinality columns of ORIGIN/DEST
df_clean = df_clean.drop(columns=['ORIGIN', 'DEST'], errors='ignore')

# Drop FL_DATE after extracting Month and Day
df_clean = df_clean.drop(columns=['FL_DATE'], errors='ignore')

print('Created Features:')
print(f'MONTH: {sorted(df_clean['MONTH'].unique())}')
print(f'HOUR: range{df_clean['HOUR'].min()}-{df_clean['HOUR'].max()}')
print(f'DAY_OF_WEEK: {df_clean['DAY_OF_WEEK'].unique()}')
print(f'OP_UNIQUE_CARRIER: {df_clean["OP_UNIQUE_CARRIER"].unique()}')
print(f'DEP_TIME_BLK: {df_clean["DEP_TIME_BLK"].unique()}')
print(f'ORIGIN_FREQ: {df_clean["ORIGIN_FREQ"].min()}-{df_clean["ORIGIN_FREQ"].max()}')
print(f'DEST_FREQ: {df_clean["DEST_FREQ"].min()}-{df_clean["DEST_FREQ"].max()}')

# Check for any missing values
missing = df_clean.isnull().sum()
missing = missing[missing >0]

if len(missing) > 0:
  print('Missing values found')
else:
  print('No missing values found')

Created Features:
MONTH: [np.int32(5), np.int32(6), np.int32(7)]
HOUR: range0-24
DAY_OF_WEEK: [1 2 3 4 5 6 7]
OP_UNIQUE_CARRIER: ['AA', 'AS', 'B6', 'DL', 'F9', ..., 'OH', 'OO', 'UA', 'WN', 'YX']
Length: 14
Categories (14, object): ['AA', 'AS', 'B6', 'DL', ..., 'OO', 'UA', 'WN', 'YX']
DEP_TIME_BLK: ['0001-0559', '0600-0659', '0700-0759', '1000-1059', '1300-1359', ..., '1800-1859', '0800-0859', '1900-1959', '2200-2259', '2300-2359']
Length: 19
Categories (19, object): ['0001-0559', '0600-0659', '0700-0759', '0800-0859', ..., '2000-2059',
                          '2100-2159', '2200-2259', '2300-2359']
ORIGIN_FREQ: 25-86562
DEST_FREQ: 25-86462
No missing values found


**Train/Test Split for Regression Dataset**

In [ ]:
y_reg = df_clean['ARR_DELAY_NEW']
X_reg = df_clean.drop(columns=['ARR_DELAY_NEW','ARR_DEL15'], errors='ignore')

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=37
)

print("Train size:", X_train_reg.shape)
print("Test size:", X_test_reg.shape)

print("Regression split complete.")

Train size: (1447885, 17)
Test size: (361972, 17)
Regression split complete.


**Train/Test Split for Classification Dataset**

In [ ]:
# Create classification X and y
X_clf_full = df_clean.drop(columns=['ARR_DEL15'])
y_clf_full = df_clean['ARR_DEL15']

# Split classification X and y into train and test
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf_full, y_clf_full,
    test_size=0.2,
    stratify=y_clf_full,
    random_state=37
)

# Combine train set for sampling
train_df = X_train_clf.copy()
train_df['ARR_DEL15'] = y_train_clf.values

delayed = train_df[train_df['ARR_DEL15'] == 1]
ontime = train_df[train_df['ARR_DEL15'] == 0]

# Downsample majority (on time)
ontime_down = ontime.sample(n=len(delayed), random_state=37)

train_balanced = pd.concat([delayed, ontime_down], ignore_index=True)
train_balanced = train_balanced.sample(frac=1, random_state=37).reset_index(drop=True)

# Split train sets again
X_train_clf = train_balanced.drop(columns=['ARR_DEL15'])
y_train_clf = train_balanced['ARR_DEL15']

print("Balanced train distribution:")
print(y_train_clf.value_counts(normalize=True))

print("Unchanged test distribution:")
print(y_test_clf.value_counts(normalize=True))

Balanced train distribution:
ARR_DEL15
1.0    0.5
0.0    0.5
Name: proportion, dtype: float64
Unchanged test distribution:
ARR_DEL15
0.0    0.730667
1.0    0.269333
Name: proportion, dtype: float64


**Save separate train/test splits for classification and regression models**

In [ ]:
save_dir = '/content/drive/MyDrive/MIS 545 Project/'

# Save classification splits
X_train_clf.to_csv(save_dir + 'X_train_clf.csv', index=False)
X_test_clf.to_csv(save_dir + 'X_test_clf.csv', index=False)
y_train_clf.to_csv(save_dir + 'y_train_clf.csv', index=False)
y_test_clf.to_csv(save_dir + 'y_test_clf.csv', index=False)

In [ ]:
#Save regression splits

X_train_reg.to_csv(save_dir + 'X_train_reg.csv', index=False)
X_test_reg.to_csv(save_dir + 'X_test_reg.csv', index=False)
y_train_reg.to_csv(save_dir + 'y_train_reg.csv', index=False)
y_test_reg.to_csv(save_dir + 'y_test_reg.csv', index=False)

**Saved preprocessor for regression:**

In [ ]:
numeric_features = [
    'CRS_DEP_TIME',
    'CRS_ELAPSED_TIME',
    'DISTANCE',
    'MONTH',
    'DAY_OF_WEEK',
    'HOUR',
    'ORIGIN_FREQ',
    'DEST_FREQ
]
categorical_features = [
    'OP_UNIQUE_CARRIER',
    'DEP_TIME_BLK'
]

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='drop'
)

save_path = '/content/drive/MyDrive/MIS 545 Project/preprocessor_final.joblib'
joblib.dump(preprocessor, save_path)

print(f"Preprocessor saved to: {save_path}")

Preprocessor saved to: /content/drive/MyDrive/MIS 545 Project/preprocessor_final.joblib
